# Image Preprocessing and Binary Classification with Keras

## Objective
In this week's exercise, you will:
1. Learn how to image preprocessing in keras.
2. Build and train a multilayer neural network for binary classification on a real-world dataset of cats and dogs.

---

## Step 1: Import Libraries
Let's start by importing the necessary libraries.


In [1]:
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


---

## Step 2: Load and Preprocess the Data
We will use the Keras `ImageDataGenerator` for image augmentation and preprocessing.
First, unzip the uploaded dataset.


In [2]:
!unzip -q kagglecatsanddogs_5340.zip

## Step 3: Learn about undersampling and implement it
Research online what undersampling and random undersampling is. It is a very powerful technique used often in machine Learning. Find out when it is used and undersample your dataset using "random undersampling"

In [3]:
# undersample your dataset here
data_dir = pathlib.Path("/content/PetImages")

# Load and preprocess the dataset
image_size = (100, 100)  # Match model input size
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size)

# Normalize the dataset
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Collect all training images and labels for undersampling
all_images = []
all_labels = []

try:
    for images, labels in train_ds:
        all_images.append(images.numpy())
        all_labels.append(labels.numpy())
except Exception as e:
    print("error", e)

# Collect all training images and labels for undersampling
val_images = []
val_labels = []

try:
    for images, labels in val_ds:
        val_images.append(images.numpy())
        val_labels.append(labels.numpy())
except Exception as e:
    print("error", e)

all_images = np.concatenate(all_images, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Flatten images for undersampling
n_samples, height, width, channels = all_images.shape
reshaped_images = all_images.reshape(n_samples, -1)

# Perform undersampling
undersampler = RandomUnderSampler(random_state=42)
sampled_images, sampled_labels = undersampler.fit_resample(reshaped_images, all_labels)

# Reshape back to image dimensions
sampled_images = sampled_images.reshape(-1, height, width, channels)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
error {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input is empty.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext] name: 
error {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Number of channels inherent in the image must be 1, 3 or 4, was 2
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext] name: 


---

## Step 4: Set Up ImageDataGenerator (or well more specifically the new version)
Were Sorry - the videos from the coursera course are sometimes not the most up to date. In this case the 'ImageDataGenerator' function is deprecated (look here https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) and will be removed in the future versions. The concept behind the new reccomended function is very similar though.
The new reccomendation is loading images with tf.keras.utils.image_dataset_from_directory and transforming the output tf.data.Dataset with preprocessing layers.

You may use Chat GPT for this task and you can also check the following tutorials <br>
https://www.tensorflow.org/tutorials/load_data/images <br>
https://www.tensorflow.org/tutorials/load_data/images <br>
https://www.tensorflow.org/guide/keras/preprocessing_layers <br>

In [ ]:
# TODO create a dataset using the recommended methods

---

## Step 5: Build a Multilayer Neural Network
Now, let's build a multilayer neural network for binary classification.


In [4]:
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(100, 100, 3)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 98, 98, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 49, 49, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 47, 47, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 21, 21, 16)          │           4,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 10, 10, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         409,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             514 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 435,250 (1.66 MB)

 Trainable params: 435,250 (1.66 MB)

 Non-trainable params: 0 (0.00 B)

---

## Step 6: Train the Model
Train the model using the Dataset you created


In [6]:
model.fit(sampled_images, sampled_labels, epochs=10,
      validation_data = (val_images, val_labels),
      verbose=2)

Epoch 1/10
223/223 - 159s - 714ms/step - accuracy: 0.5315 - loss: 0.6882 - val_accuracy: 0.4688 - val_loss: 0.6906
Epoch 2/10
223/223 - 152s - 683ms/step - accuracy: 0.5946 - loss: 0.6638 - val_accuracy: 0.5000 - val_loss: 0.6786
Epoch 3/10
223/223 - 155s - 696ms/step - accuracy: 0.6390 - loss: 0.6352 - val_accuracy: 0.5312 - val_loss: 0.6933
Epoch 4/10
223/223 - 200s - 896ms/step - accuracy: 0.6896 - loss: 0.5790 - val_accuracy: 0.6250 - val_loss: 0.7237
Epoch 5/10
223/223 - 203s - 912ms/step - accuracy: 0.7623 - loss: 0.4890 - val_accuracy: 0.5625 - val_loss: 0.7983
Epoch 6/10
223/223 - 200s - 898ms/step - accuracy: 0.8470 - loss: 0.3399 - val_accuracy: 0.5312 - val_loss: 1.0153
Epoch 7/10
223/223 - 155s - 697ms/step - accuracy: 0.9184 - loss: 0.1960 - val_accuracy: 0.6562 - val_loss: 1.5776
Epoch 8/10
223/223 - 200s - 898ms/step - accuracy: 0.9554 - loss: 0.1171 - val_accuracy: 0.6250 - val_loss: 1.8983
Epoch 9/10
223/223 - 154s - 691ms/step - accuracy: 0.9784 - loss: 0.0737 - val_a

---

## Step 7: Evaluate the Model
After training, you may upload some test images to evaluate your model.


In [13]:
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files

def load_and_predict(model):
    uploaded_files = files.upload()

    for fn in uploaded_files.keys():
        path = '/content/' + fn
        img = image.load_img(path, target_size=(100, 100))

        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0) / 255.0

        classes = model.predict(x)
        result = "a dog" if classes[0][1] > 0.5 else "a cat"

        print(f'The model predicts that the image is of {result}')

# Call the function to upload images and get predictions
load_and_predict(model)

Saving 1011.jpg to 1011.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
The model predicts that the image is of a cat
